In [34]:
!pip install -q "accelerate>=0.29.3"
!pip install -q "huggingface_hub>=0.23.0"
!pip install -q "datasets>=2.19.1"
!pip install -q transformers
# Optional:
# !pip install -q "flash-attn>=2.5.8"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/aoyagihiroki/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/aoyagihiroki/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/aoyagihiroki/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/Users/aoyagihiroki/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [35]:
model_id = "microsoft/Phi-3-mini-128k-instruct"  # @param {type:"string"}

!echo Model page: https://huggingface.co/{model_id}

Model page: https://huggingface.co/microsoft/Phi-3-mini-128k-instruct


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)


model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.71s/it]


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [37]:
from functools import cache
import re

def loglikelihood(prompt, completion):
    # LLMへの入力を定義
    text = prompt + completion

    # completionが始まる・終わるindicesを取得
    start_idx = len(tokenizer.encode(prompt, add_special_tokens=False))
    end_idx = len(tokenizer.encode(text, add_special_tokens=False))

    # print(f"start_idx: {start_idx}, end_idx: {end_idx}")

    # Xはtext（prompt + completion）をエンコードしたもの。
    X = tokenizer.encode(text, return_tensors="pt", add_special_tokens=False).to(model.device)
    # print("X: ", X)
    # print("X.shape: ", X.shape)

    # Completionと対比する部分の出力を取得
    logits = model(X[:, :-1]).logits[0, start_idx - 1 :, :]  # X[:, :-1]は、Xのすべての行で最後の列を除いたもの。
    # print(model(X[:, :-1]).logits[0, -1, :].shape)
    # print("model(X[:, :-1]): ", model(X[:, :-1]))
    # print("logits: ", logits)
    # print("model(X[:, :-1]).logits.shape: ", model(X[:, :-1]).logits.shape)
    # print("logits.shape: ", logits.shape)
    
    # print("X[0, start_idx:end_idx]: ", X[0, start_idx:end_idx])

    # # promptを条件としてcompletionが生成される対数確率を算出する
    # return -torch.nn.CrossEntropyLoss(reduction="sum")(logits, X[0, start_idx:end_idx]).item()
    # """以下と同等"""
    logprobs = logits.softmax(dim=-1).log()
    logprobs = logprobs[range(end_idx - start_idx), X[0, start_idx:end_idx]]
    formatted_list = [f"{x:.4f}" for x in logprobs.tolist()]
    # print(completion, formatted_list)
    return logprobs.sum(), logits, formatted_list



def printResult(prompt, completions):
    # 正規表現で<|user|>と<|end|>の間の文字列を抽出
    prompt_notSpecialTokens = re.search(r"<\|user\|>(.*?)<\|end\|>", prompt, re.DOTALL).group(1).strip()

    print(f"【プロンプト】{prompt_notSpecialTokens}")

    print(f"【回答候補】{completions}")
    print(f"")
    print(f"【各回答のLog likelihood】")
    for completion in completions:
        p_log, logits, ll_list = loglikelihood(prompt, completion)
        # print(p_log.tolist())


    # print(logits.shape)

    print(ll_list)
    print()
    print("------------------------")

    for i in range(len(logits)):
        print("")
        print(f"{i}番目")
        # 最後のトークンのlogitsを取得
        last_token_logits = logits[i, :]

        # logitsをsoftmaxで確率に変換
        probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)

        # 上位のトークンとその確率を取得
        top_k = 10  # 上位k個のトークンを取得
        top_k_indices = torch.topk(probabilities, top_k).indices
        # print(top_k_indices[0].tolist())
        top_k_probs = torch.topk(probabilities, top_k).values
        # print(top_k_probs[0])

        # トークンと確率の表示
        for idx, prob in zip(top_k_indices, top_k_probs):
            token = tokenizer.decode([idx.item()])
            print(f"Token: {token}, Probability: {prob.item():.4f}")

In [38]:
dict_1A = {}
dict_1B = {}
dict_2A = {}
dict_2B = {}
for i in range(10001):
    print(i)
    # テキストファイルを読み込む
    with open(f'prompt/{i}-1.txt', 'r') as file:
        content = file.read()  # ファイルの全内容を読み込む

    # print(content)
    
    _, _, ll_list = loglikelihood(f"""\
    <|user|>
    {content}<|end|>
    <|assistant|>
    """, "A")
    dict_1A[i] = ll_list[0]

    _, _, ll_list = loglikelihood(f"""\
    <|user|>
    {content}<|end|>
    <|assistant|>
    """, "B")
    dict_1B[i] = ll_list[0]


    # テキストファイルを読み込む
    with open(f'prompt/{i}-2.txt', 'r') as file:
        content = file.read()  # ファイルの全内容を読み込む

    # print(content)

    _, _, ll_list = loglikelihood(f"""\
    <|user|>
    {content}<|end|>
    <|assistant|>
    """, "A")
    dict_2A[i] = ll_list[0]

    _, _, ll_list = loglikelihood(f"""\
    <|user|>
    {content}<|end|>
    <|assistant|>
    """, "B")
    dict_2B[i] = ll_list[0]

print(dict_1A)
print(dict_1B)
print(dict_2A)
print(dict_2B)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [39]:
print(dict_1A)

{0: '-2.2723', 1: '-2.3561', 2: '-2.1295', 3: '-2.2565', 4: '-2.5427', 5: '-2.4866', 6: '-0.4037', 7: '-1.1182', 8: '-2.1787', 9: '-2.3311', 10: '-2.3439', 11: '-2.4452', 12: '-2.2512', 13: '-2.1550', 14: '-1.8068', 15: '-1.3046', 16: '-2.0141', 17: '-1.8371', 18: '-1.8557', 19: '-1.3506', 20: '-2.1736', 21: '-2.2867', 22: '-2.4004', 23: '-2.3743', 24: '-2.7493', 25: '-2.4165', 26: '-2.7871', 27: '-1.4138', 28: '-2.4025', 29: '-2.1276', 30: '-2.5022', 31: '-1.6743', 32: '-2.5814', 33: '-1.9169', 34: '-2.2408', 35: '-2.3169', 36: '-2.6469', 37: '-2.1845', 38: '-2.3851', 39: '-2.2935', 40: '-2.3233', 41: '-2.2760', 42: '-2.3665', 43: '-2.4353', 44: '-2.3680', 45: '-2.6056', 46: '-2.1751', 47: '-1.7512', 48: '-2.3817', 49: '-2.5214', 50: '-1.4714', 51: '-2.0349', 52: '-1.9099', 53: '-2.5056', 54: '-1.9584', 55: '-1.8757', 56: '-2.4885', 57: '-2.0025', 58: '-2.2878', 59: '-2.1526', 60: '-2.2573', 61: '-1.8825', 62: '-2.4351', 63: '-1.9386', 64: '-2.2641', 65: '-2.1401', 66: '-2.4374', 67: 

In [43]:
print(dict_1A[10000])

-2.1277


In [45]:
import pandas as pd

# データを格納するためのリストを初期化
data = {
    'll_1A': [],
    'll_1B': [],
    'll_2A': [],
    'll_2B': []
}

# ループを使って各辞書から値を取得
for i in range(10000):
    data['ll_1A'].append(dict_1A[i])
    data['ll_1B'].append(dict_1B[i])
    data['ll_2A'].append(dict_2A[i])
    data['ll_2B'].append(dict_2B[i])

# データフレームを作成
df = pd.DataFrame(data)

# CSVに書き込み
df.to_csv('loglikelihood_10000.csv', index=False)